In [9]:
import pandas as pd
import xgboost as xgb
import os
import logging
import time
import cPickle as pickle
import numpy as np

# from IPython.utils.io import Tee
# wtf = Tee('../logs/%s.log' % str(int(time.time())), 'w', channel='stderr')

In [ ]:
%%time
PATH_STORE = '../saved/storage.h5'
train_pd = pd.read_hdf(PATH_STORE, 'train')

In [ ]:
%%time
# Out-of-memory
PATH_LIBSVM = '/tmp/train.libsvm'
PATH_CACHE = '/tmp/dtrain.cache'
if os.path.exists(PATH_CACHE):
    print 'Remove old cache @ %s' % PATH_CACHE
    os.remove(PATH_CACHE)
    os.remove(PATH_CACHE + '.row.blob')

# dtrain = xgb.DMatrix('../saved/med.libsvm#%s' % PATH_CACHE)
dtrain = xgb.DMatrix('%s#%s' % (PATH_LIBSVM, PATH_CACHE))

In [2]:
%%time
# In-memory
PATH_LIBSVM = '/tmp/train.libsvm'
dtrain = xgb.DMatrix(PATH_LIBSVM)

CPU times: user 40.1 s, sys: 1.56 s, total: 41.6 s
Wall time: 20.1 s


In [25]:
params = {}
params['objective'] = "binary:logistic"
params['eta'] = 0.01
params['min_child_weight'] = 6
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7
params['max_delta_step'] = 1
params['silent'] = 1
params['max_depth'] = 9 #7
params['eval_metric'] = 'auc'
# params["scale_pos_weight"] = 1.0
params['gamma'] = 5
params['alpha'] = 1
# params['nthread'] = 4

def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label==1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

plst = list(params.items())

max_rounds = 4000


In [ ]:
%%time
cv_log = xgb.cv(params, dtrain, max_rounds, nfold=3,
                metrics={'auc'}, seed = 0,
                fpreproc=fpreproc)

pickle.dump((params, cv_log), open('../saved/cv_log_%d.p' % int(time.time()), 'wb'))

In [5]:
y = dtrain.get_label()

In [6]:
from collections import Counter
Counter(y)

Counter({0.0: 111415, 1.0: 33760})

In [24]:
for l in cv_log[-500::10]:
    print l

[3500]	cv-test-auc:0.790100+0.001692	cv-train-auc:0.999698+0.000007
[3510]	cv-test-auc:0.790126+0.001696	cv-train-auc:0.999705+0.000007
[3520]	cv-test-auc:0.790124+0.001705	cv-train-auc:0.999711+0.000006
[3530]	cv-test-auc:0.790134+0.001706	cv-train-auc:0.999717+0.000006
[3540]	cv-test-auc:0.790146+0.001703	cv-train-auc:0.999721+0.000006
[3550]	cv-test-auc:0.790147+0.001706	cv-train-auc:0.999727+0.000007
[3560]	cv-test-auc:0.790147+0.001691	cv-train-auc:0.999733+0.000007
[3570]	cv-test-auc:0.790154+0.001695	cv-train-auc:0.999737+0.000008
[3580]	cv-test-auc:0.790160+0.001705	cv-train-auc:0.999743+0.000008
[3590]	cv-test-auc:0.790165+0.001701	cv-train-auc:0.999748+0.000007
[3600]	cv-test-auc:0.790157+0.001704	cv-train-auc:0.999753+0.000007
[3610]	cv-test-auc:0.790173+0.001696	cv-train-auc:0.999758+0.000007
[3620]	cv-test-auc:0.790186+0.001687	cv-train-auc:0.999763+0.000006
[3630]	cv-test-auc:0.790205+0.001687	cv-train-auc:0.999767+0.000006
[3640]	cv-test-auc:0.790204+0.001679	cv-train-au

# Train on whole set

In [ ]:
bst = xgb.train(params, dtrain, max_rounds)
bst.save_model('%d.model' % int(time.time()))

# Predict

In [30]:
# In-memory
PATH_TEST_LIBSVM = '/tmp/test.libsvm'
dtest = xgb.DMatrix(PATH_TEST_LIBSVM)

In [31]:
preds = bst.predict(dtest)

# Create submission

In [38]:
sub = pd.read_csv('../data/sample_submission.csv', index_col='ID')
sub['target'] = preds
sub.to_csv('../submissions/xgb_vanilla.csv')